In [1]:
import pandas as pd
import numpy as np
import tqdm

from pathlib import Path
datadir = Path("./")

# Load DataFrames with embedding ids and texts
df_acts_filename = datadir / "tinystories1M-TinyStories1_gpt2token-acts_id.csv"
df_texts_filename = datadir / "tinystories1M-TinyStories1_gpt2token-texts.csv"
df_acts_tokens = pd.read_csv(df_acts_filename)
df_texts= pd.read_csv(df_texts_filename)

# Load embedding matrix
with open(datadir / 'emb_matrix.npy', 'rb') as f:
    embs_flat = np.load(f)
    
# Left join activations dataframe with sequence dataframe
df_acts_tokens_texts = df_acts_tokens.merge(df_texts, how='left', on='seq_id')
df_acts_tokens_texts



,emb_id,token_str,seq_id,seq
0,0,One,0,"One day, a little girl named Lily found a need..."
1,1,day,0,"One day, a little girl named Lily found a need..."
2,2,",",0,"One day, a little girl named Lily found a need..."
3,3,a,0,"One day, a little girl named Lily found a need..."
4,4,little,0,"One day, a little girl named Lily found a need..."
...,...,...,...,...
682590,682590,very,3200,Once there was a little boy named Timmy. He wa...
682591,682591,proud,3200,Once there was a little boy named Timmy. He wa...
682592,682592,of,3200,Once there was a little boy named Timmy. He wa...
682593,682593,him,3200,Once there was a little boy named Timmy. He wa...


In [ ]:
# Slow and not optimal but working

import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_std=False)
Xs = scaler.fit_transform(embs_flat)

W_oja  = np.random.rand(Xs.shape[1], 1)
prev_W_oja = np.ones((Xs.shape[1], 1))

epoch = 0
learning_rate = 0.01
tolerance = 1e-8

while np.linalg.norm(prev_W_oja - W_oja) > tolerance:
    prev_W_oja = W_oja.copy()
    
    
    for xs in Xs:
        xs = xs.reshape((Xs.shape[1], 1))
        Ys = np.dot(W_oja.T, xs)
        W_oja += learning_rate * Ys * (xs - Ys*W_oja)
    epoch+=1

print(f"Converged in {epoch} epochs")
print("First principal component:", W_oja.flatten())

In [ ]:
ipc = 0
name = f"pc{ipc}_oja"

Q = embs_flat @ W_oja
df_acts_tokens[name] = Q
df_acts_tokens.sort_values(name, ascending=False).head(40)

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler(with_std=False)
Xs = scaler.fit_transform(embs_flat)  

learning_rate = 0.001
n_features = Xs.shape[1]
n_components = 10
tolerance = 1e-6 


W_sanger = np.random.normal(scale=0.1, size=(n_components, n_features))
prev_W_sanger = np.ones((n_components, n_features))

t = 0.0  
for iteration in tqdm(range(10)):
    t += 1.0
    dw = np.zeros_like(W_sanger)  

    for j in range(Xs.shape[0]):
        Ysj = np.dot(W_sanger, Xs[j])  
        QYd = np.tril(np.outer(Ysj, Ysj))  
        dw += np.outer(Ysj, Xs[j]) - np.dot(QYd, W_sanger)  

    prev_W_sanger = W_sanger.copy()
    W_sanger += (learning_rate / t) * dw

    # Нормализация весов
    W_sanger /= np.linalg.norm(W_sanger, axis=1, keepdims=True)

    # Проверка сходимости
    if np.linalg.norm(W_sanger - prev_W_sanger) < tolerance:
        print(f"Converged at {iteration + 1} iteration")
        break


In [ ]:
ipc = 2
name = f"pc{ipc}_sanger"

Q = embs_flat @  W_sanger[ipc]
df_acts_tokens[name] = Q
df_acts_tokens.sort_values(name, ascending=False).head(40)

In [ ]:
from sklearn.decomposition import FastICA, PCA
scaler = StandardScaler()
embs_scaled = scaler.fit_transform(embs_flat)


pca = PCA(with_std=False)  
embs_pca = pca.fit_transform(embs_scaled)

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)


optimal_components = np.argmax(cumulative_variance >= 0.95) + 1

pca_optimal = PCA(n_components=optimal_components)
embs_pca_optimal = pca_optimal.fit_transform(embs_scaled)

ipc = 2
name = f"pc{ipc}"
Q = embs_flat@ pca_optimal.components_[ipc]
df_acts_tokens[name] = Q
df_acts_tokens.sort_values(name, ascending=False).head(40)